In [1]:
from pypokerengine.api.game import setup_config, start_poker
from pypokerengine.players import BasePokerPlayer
from baseline_players import RandomPlayer
# from bots.honest_player import HonestPlayer
# from baseline_players import HonestPlayer
from bots.fish_player import FishPlayer
from bots.fold_player import FoldPlayer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [2]:
N_RANDOM_PLAYERS = 8

config = setup_config(max_round=50, initial_stack=1500, small_blind_amount=15)
config.register_player(name="fishman", algorithm=FishPlayer())
# config.register_player(name="foldman", algorithm=FoldPlayer())
# config.register_player(name="honest", algorithm=HonestPlayer(20))
for i in range(N_RANDOM_PLAYERS):
    config.register_player(name="random" + str(i), algorithm=RandomPlayer())

In [3]:
%%time

def get_all_results(config, n_games=3):
    players_stack_dict = {}
    for player_info in config.players_info:
        players_stack_dict[player_info['name']] = []
        
    for i in tqdm(range(n_games)):
        for player_info in start_poker(config, verbose=0)['players']:
            players_stack_dict[player_info['name']].append(player_info['stack'])
            
    return pd.DataFrame(players_stack_dict)

res = get_all_results(config, n_games=2)
#3.4s/it 5:40 - my
#5.9s/it 9:59 - base

100%|██████████| 2/2 [00:00<00:00,  8.82it/s]

CPU times: user 225 ms, sys: 4.36 ms, total: 230 ms
Wall time: 231 ms


In [4]:
res.describe()

,fishman,random0,random1,random2,random3,random4,random5,random6,random7
count,2.000000,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
mean,13424.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,28.991378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,13404.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,13414.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,13424.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,13434.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,13445.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
% time np.array([np.random.choice(52, 7, replace=False) for _ in range(1000)]) + 1

CPU times: user 35 ms, sys: 7.06 ms, total: 42.1 ms
Wall time: 36.9 ms


array([[46, 50, 49, ...,  6, 10, 45],
       [14, 41, 32, ..., 10,  9, 42],
       [11, 20, 22, ..., 28, 12, 48],
       ..., 
       [ 4, 20, 22, ..., 39, 52, 33],
       [41, 20, 14, ...,  5, 45, 11],
       [44,  3, 18, ..., 13, 50,  5]])